# Medical MNIST

 - - -

## Dataset Instructions

1. ChestMNIST: 
> Chest X-ray dataset \
> Multi-Label(14) & Binary-Class(2) \
> 112,120 samples
2. BreastMNIST: 
> Breast UltraSound \
> Binary-Class(2) \
> 780 samples

- - -

In [ ]:
import os
import time
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torch.nn as nn

In [ ]:
from scipy.stats.stats import pearsonr
from scipy.stats import ttest_ind
from scipy.stats import bartlett
from scipy.stats import ks_2samp
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier

from torch.nn import functional as F
from torch.autograd import Variable
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, TensorDataset

In [ ]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
class Args:
    # arugments
    epochs=50
    bs=32
    lr=0.001
    momentum=0.9
    num_classes=3
    verbose='store_true'
    seed=674

args = Args()    

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)

In [ ]:
#Setting torch environment

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device: ', DEVICE)

- - -

# Data Handling

## Uploading dataset

In [ ]:
# Data Transformation
data_transforms = transforms.Compose([
    transforms.Resize(256),
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(0.5),
#     transforms.ColorJitter(brightness=(0.2, 2), 
#                                contrast=(0.3, 2), 
#                                saturation=(0.2, 2), 
#                                hue=(-0.3, 0.3)),
    transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Uploading the food image data
# lung_data = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image/LUNG_public/COVID-19_Radiography_Dataset', transform = data_transforms)
lung_pneu_train = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image/LUNG_pneumonia/train', transform = data_transforms)
lung_pneu_test  = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image/LUNG_pneumonia/test', transform = data_transforms)

In [ ]:
lung_pneu_train

- - -

## Data preprocessing

In [ ]:
## scaler setting for data standardization.
scaler = MinMaxScaler()

In [ ]:
train_loader = torch.utils.data.DataLoader(lung_pneu_train, batch_size=args.bs, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(lung_pneu_test, batch_size=args.bs, shuffle=False, num_workers=4)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(labels)

## Calculating Mean and Std

In [ ]:
def normalize_image(train_loader):

    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)

    for images, _ in train_loader:

        b, c, h, w = images.shape
        nb_pixels = b * h * w
        sum_ = torch.sum(images, dim=[0, 2, 3])
        sum_of_square = torch.sum(images ** 2, dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)

        cnt += nb_pixels

    return fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)

In [ ]:
lung_mean, lung_std = normalize_image(train_loader)

In [ ]:
print(lung_mean, lung_std)

* Based on calculated mean and standard deviation, upload the dataset again with new data_transform

In [ ]:
# Data Transformation
data_transforms_st = transforms.Compose([
    transforms.Resize((256,256)),
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(0.5),
#     transforms.ColorJitter(brightness=(0.2, 2), 
#                                contrast=(0.3, 2), 
#                                saturation=(0.2, 2), 
#                                hue=(-0.3, 0.3)),
    transforms.ToTensor(),
    transforms.Normalize([0.5535, 0.5535, 0.5535], [0.1895, 0.1895, 0.1895])
])

In [ ]:
# Uploading the food image data
# lung_data = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image/LUNG_public/COVID-19_Radiography_Dataset', transform = data_transforms)
lung_pneu_train = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image/LUNG_pneumonia/train', transform = data_transforms_st)
lung_pneu_test  = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/image/LUNG_pneumonia/test', transform = data_transforms_st)

In [ ]:
train_loader = torch.utils.data.DataLoader(lung_pneu_train, batch_size=args.bs, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(lung_pneu_test, batch_size=args.bs, shuffle=False, num_workers=4)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()
print(labels)
print(labels.size())

In [ ]:
classes = lung_pneu_train.classes
print(classes)

- - -

# Data Visualization

In [ ]:
images[0].shape

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(labels)
print(labels.size())

In [ ]:
plt.imshow(images[0].permute(1,2,0))
plt.title("Normal")
plt.show()

- - -

# Statistical Approaches

* Concept: How about generating additional dataset from current limited dataset, based on statistical theories?
> 1. Check the distribution of each data features(SDNN, ...) and visualize.
> 2. Calculate correlation coefficients between variables based on regression.
> 3. Calculate their mean, sd, and other statistics to find out its distribution.
>> However, most of them would be from normal distribution with different μ and σ based on the CLT.
> 4. Generate random dataset based on its distribution, correlation, and regression coefficients.

## HRV Variable Distributions

* dataset lists:
> baseline1 ~ stress  -- hrv_b1_s_sub \
> stress ~ baseline2  -- hrv_s_b2_sub \
> baseline2 ~ rest    -- hrv_b2_r_sub \
> rest ~ baseline3    -- hrv_r_b3_sub \
> baseline3 ~ recovery -- hrv_b3_c_sub \
> stress ~ rest  -- hrv_s_r_sub

In [ ]:
data_vis = hrv_s_r_sub

* Generating new dataframe that we want to check the distribution of.

In [ ]:
hrv_visual = pd.concat([data_vis, hrv_disorder],axis=1)

In [ ]:
## Separating dataframe into three different groups (CONTROL, MDD, PD)
hrv_CON = hrv_visual[hrv_visual["disorder"] == 2]
hrv_MDD = hrv_visual[hrv_visual["disorder"] == 0]
hrv_PD = hrv_visual[hrv_visual["disorder"] == 1]

* Total 13 variables: SDNN, NN50, PNN50, RMSSD, VLF, LF, HF, LF/HF, POWER, HR, RESP, SC, TEMP

In [ ]:
## Set the variable that we want to check
var = "SC"

In [ ]:
CON = hrv_CON[var]
MDD = hrv_MDD[var]
PD = hrv_PD[var]

* Comparing one variable for three groups

In [ ]:
plt.figure(figsize = (10,5))
sns.set_style("whitegrid")
plt.grid(True)
plt.xlabel('Variable: LF/HF ratio',fontsize=10)
plt.ylabel('Density',fontsize=10)

sns.kdeplot(CON)
sns.kdeplot(MDD)
sns.kdeplot(PD)

# plt.legend()
plt.legend(['Control', 'Major Depressive Disorder', 'Panic Disorder'], fontsize=10)

# plt.savefig('./data/figures/distributions/stress_rest/TEMP.png')

* Distribution check (based on statistics)

In [ ]:
## T-test for equal mean value check
## if p-value < 0.05, two distributions do not have equal mean values.
print(">T-TEST")
print("Mean value check for CON and MDD, p-value: {:.3f}".format(ttest_ind(CON, MDD).pvalue))
print("Mean value check for CON and PD, p-value: {:.3f}".format(ttest_ind(CON, PD).pvalue))
print("Mean value check for MDD and PD, p-value: {:.3f}".format(ttest_ind(MDD, PD).pvalue))
print("-----------------------------------------------")


## Bartlett-test for equal variability check
## if p-value < 0.05, two distributions do not have equal variance.
print(">Bartlett-test")
print("Equal Variability test for CON and MDD, p-value: {:.3f}".format(bartlett(CON, MDD).pvalue))
print("Equal Variability test for CON and PD, p-value: {:.3f}".format(bartlett(CON, PD).pvalue))
print("Equal Variability test for MDD and PD, p-value: {:.3f}".format(bartlett(MDD, PD).pvalue))
print("-----------------------------------------------")


## Shapiro-Wilk test for normal distribution check
## if p-value < 0.05, distribution is not following normal distribution.
print(">Shapiro-Wilks test")
print("Normal distribution test for CON, p-value: {:.3f}".format(shapiro(CON).pvalue))
print("Normal distribution test for MDD, p-value: {:.3f}".format(shapiro(MDD).pvalue))
print("Normal distribution test for PD, p-value: {:.3f}".format(shapiro(PD).pvalue))
print("-----------------------------------------------")


## Kolmogorov-Smirnov test for equal distribution check
## if p-value < 0.05, two distributions are not following same distribution. 
print(">Kolmogorov-Smirnov test")
print("Equal distributions test between CON and MDD, p-value: {:.3f}".format(ks_2samp(CON, MDD).pvalue))
print("Equal distributions test between CON and PD, p-value: {:.3f}".format(ks_2samp(CON, PD).pvalue))
print("Equal distributions test between MDD and PD, p-value: {:.3f}".format(ks_2samp(MDD, PD).pvalue))
print("-----------------------------------------------")

In [ ]:
SDNN = hrv_only['b1SDNN']
NN50 = hrv_only['b1NN50']
PNN50 = hrv_only['b1PNN50']
RMSSD = hrv_only['b1RMSSD']
VLF = hrv_only['b1VLF']
LF = hrv_only['b1LF']
HF = hrv_only['b1HF']
LFHF = hrv_only['b1LF/HF']
POWER = hrv_only['b1POWER']
RESP = hrv_only['b1RESP']
TEMP = hrv_only['b1TEMP']
HR = hrv_only['b1HR']

* All variables

In [ ]:
plt.figure(figsize = (10,5))
sns.set_style("whitegrid")
plt.grid(True)
plt.xlabel('Standardized Variables',fontsize=10)
plt.ylabel('Density',fontsize=10)

sns.kdeplot(b1SDNN)
sns.kdeplot(b1NN50)
sns.kdeplot(b1RMSSD)
# sns.kdeplot(b1VLF)
sns.kdeplot(b1LF)
# sns.kdeplot(b1HF)
sns.kdeplot(b1LFHF)
# sns.kdeplot(b1POWER)
# sns.kdeplot(b1PNN50)
sns.kdeplot(b1RESP)
sns.kdeplot(b1TEMP)
sns.kdeplot(b1HR)

# plt.legend()
plt.legend(['b1SDNN', 'b1NN50', 'b1RMSSD', 'b1LF', 'b1LF/HF', 'b1RESP', 'b1TEMP', 'b1HR'], fontsize=10)

- - -

## Central Limit Theorem approach

- - -

## Correlation between data features

* To generate new dataset from each feature distribution, we have to realize the correlation and regression coefficients.

In [ ]:
hrv_visual.columns

In [ ]:
hrv_visual.corr()

* Visualize the correlation

In [ ]:
plt.figure(figsize = (15,15))
corrMat = hrv_visual.corr()
sns.heatmap(corrMat, annot=True)
plt.show()

* Check whether each correlation coefficient is reliable

In [ ]:
## pearsonr function shows individual correlation coefficient with p-value
pearsonr(hrv_visual['SDNN'], hrv_visual['NN50'])

In [ ]:
## for loop to calculate correlation coefficient and following p-values for every variables.
col = list(hrv_visual)
corr_result = []
for i in range(0,len(col)-1):
    a = hrv_visual[hrv_visual.columns[i]]
    i += 1
    b = hrv_visual[hrv_visual.columns[i]]
    cor = pearsonr(a, b)
    corr_result.append(cor)

In [ ]:
corr_result_df = pd.DataFrame(corr_result, columns=['correlation', 'p-value'])

In [ ]:
var_names = []
for i in range(0,len(col)-1):
    cur_var = (col[i], col[i+1])
    var_names.append(cur_var)

In [ ]:
var_names_df = pd.DataFrame(var_names, columns=['Variable #1', 'Variable #2'])

In [ ]:
correlation_df = pd.concat([var_names_df, corr_result_df], axis=1)

In [ ]:
correlation_df['reliability'] = np.where(correlation_df['p-value']<0.05, "o", "x")

In [ ]:
correlation_df

In [ ]:
sd = np.std(hrv_visual['SDNN'])

In [ ]:
hrv_visual.mean()

In [ ]:
hrv_visual.std()

## Regression Coefficients

* To generate new dataset from each feature distribution, we have to realize the correlation and regression coefficients.

In [ ]:
hrv_visual.columns

In [ ]:
# features = hrv_visual[['SDNN', 'NN50', 'PNN50', 'RMSSD', 'VLF', 'LF', 'HF', 'LF/HF', 'POWER', 'HR', 'RESP', 'SC', 'TEMP']]
# features = hrv_visual[['SDNN', 'NN50', 'PNN50', 'RMSSD', 'LF/HF', 'HR']]
features = hrv_visual[['PNN50', 'LF/HF', 'HR']] ## variables that mentioned from previous research.(professor Jeon.)

disorder = hrv_visual[['disorder']]

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, disorder)

In [ ]:
model = LogisticRegression()
model.fit(train_features, train_labels)

In [ ]:
print(model.score(train_features, train_labels))

In [ ]:
print(model.coef_)

- - -

# Data Analysis

## Autoencoder

* Here, we are going to use autoencoder algorithm to effectively extract the core features from dataset
* Autoencoder is useful for reducing high-dimensionality dataset

In [ ]:
print(len(train_loader))
print(len(test_loader))

### Autoencoder Model

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 3 --> 16), with 3x3 kernels
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2)

    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # compressed representation
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x

In [ ]:
# initialize the NN
model = ConvAutoencoder()
print(model)

In [ ]:
# class Autoencoder(nn.Module):
#     def __init__(self):
#         super(Autoencoder, self).__init__()
        
#         ## encoder is similar to the simple neural network
#         self.encoder = nn.Sequential(
#             nn.Linear(28*28, 128), # gradually reducing dimensionality
#             nn.ReLU(),
#             nn.Linear(128, 64),
#             nn.ReLU(),
#             nn.Linear(64, 12),
#             nn.ReLU(),
#             nn.Linear(12, 3)
#         )
#         ## decoder is recovering the dimensionality to origianl dataset size
#         self.decoder = nn.Sequential(
#             nn.Linear(3, 12), # gradually increasing dimensionality
#             nn.ReLU(),
#             nn.Linear(12, 64),
#             nn.ReLU(),
#             nn.Linear(64, 128),
#             nn.ReLU(),
#             nn.Linear(128, 28*28),
#         )
        
#     def forward(self, x):
#         encoded = self.encoder(x)         ## creating latent varialbe 'encoder'
#         decoded = self.decoder(encoded)   ## generating recovered image 'decoded'
#         return encoded, decoded

### Loss and optimization function

In [ ]:
autoencoder = Autoencoder().to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr = args.lr)  ## Adam for optimization function.
criterion = nn.MSELoss()  ## Using MSE(Mean Squared Error) to calculate the differences between original data and decoded data